In [2]:
#imports 

# from sklearn.datasets import load_boston
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import mean_squared_error

# import pandas as pd
# import numpy as np

from sklearn.datasets import load_boston
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import pandas as pd
import numpy as np

In [5]:
#gather data

boston_dataset = load_boston()
# boston_dataset.describe()
type(boston_dataset)
# to convert the dataset into a panda dataframe

data = pd.DataFrame(data = boston_dataset.data, columns = boston_dataset.feature_names)
features = data.drop(['INDUS', 'AGE'], axis = 1)

data.describe()
boston_dataset.feature_names

#to make the data set of the prices
log_prices = np.log(boston_dataset.target)
target = pd.DataFrame(log_prices, columns = ['PRICE'])

# target.head()
# boston_dataset.target

In [6]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [7]:
CRIME_IDX = 0
ZN_IDX = 1
CHAS_IDX = 2
RM_IDX = 4
PTRATIO_IDX = 8

property_stats = features.mean().values.reshape(1,11)
property_stats

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [8]:
type(features.mean())

pandas.core.series.Series

In [9]:
type(features.mean().values)

numpy.ndarray

In [10]:
type(features.mean().values.reshape(1,11))

numpy.ndarray

In [11]:
features.mean().values.reshape(1,11)

array([[3.61352356e+00, 1.13636364e+01, 6.91699605e-02, 5.54695059e-01,
        6.28463439e+00, 3.79504269e+00, 9.54940711e+00, 4.08237154e+02,
        1.84555336e+01, 3.56674032e+02, 1.26530632e+01]])

In [13]:
#training the model with linear regression with target prices over the features dataset
regr = LinearRegression().fit(features, target)

In [15]:
fitted_values = regr.predict(features)
fitted_values

array([[3.39664677],
       [3.19835664],
       [3.41211691],
       [3.36926572],
       [3.31497108],
       [3.25264829],
       [3.108364  ],
       [2.91044427],
       [2.5413092 ],
       [2.91620602],
       [2.86872454],
       [3.04985366],
       [3.00689598],
       [3.00900293],
       [2.97077174],
       [3.00286629],
       [3.05806467],
       [2.84683392],
       [2.8669729 ],
       [2.94614728],
       [2.64187504],
       [2.88297883],
       [2.75886338],
       [2.69127888],
       [2.79037709],
       [2.71360592],
       [2.80193695],
       [2.73229701],
       [2.93654401],
       [2.98183279],
       [2.58162964],
       [2.89515735],
       [2.41421346],
       [2.72436498],
       [2.65432884],
       [3.16700996],
       [3.09994804],
       [3.15567482],
       [3.12937335],
       [3.42933686],
       [3.53510293],
       [3.33812743],
       [3.25490029],
       [3.21647699],
       [3.14153317],
       [3.12224447],
       [3.01880036],
       [2.875

In [16]:
#calculating the mean squared error and the rmse
MSE = mean_squared_error(target, fitted_values)
MSE

0.03516080084618688

In [17]:
#calculating the rmse
RMSE = np.sqrt(MSE)
RMSE

0.18751213519713034

In [20]:
def get_log_estimate(nr_rooms,
                    student_per_classroom,
                    next_to_river = False,
                    high_confidence = True):
    # Configure property
    property_stats[0][RM_IDX] = nr_rooms
    property_stats[0][PTRATIO_IDX] = student_per_classroom
    
    if next_to_river:
        property_stats[0][CHAS_IDX] = 1
    else:
        property_stats[0][CHAS_IDX] = 0
    
    # Make prediction
    log_estimate = regr.predict(property_stats)[0][0]
    
    # Calc Range 
    if high_confidence:
        upper_bound = log_estimate + 2*RMSE
        lower_bound = log_estimate - 2*RMSE
        interval = 95
    else:
        upper_bound = log_estimate + RMSE
        lower_bound = log_estimate - RMSE
        interval = 68
    
    return log_estimate, upper_bound, lower_bound, interval

In [21]:
get_log_estimate(3,20,next_to_river=True, high_confidence=False)

(2.776758191480399, 2.9642703266775294, 2.589246056283269, 68)

In [22]:
np.median(boston_dataset.target)

21.2

In [27]:
#python code that convert the log prices to the dollar value

ZILLOW_MEDIAN_PRICE = 583.3
SCALE_FACTOR= ZILLOW_MEDIAN_PRICE/np.median(boston_dataset.target)

log_est, upper, lower, conf = get_log_estimate(9,15,
                                               next_to_river=False, 
                                               high_confidence=False)

#converting the price to dollar
dollar_est = np.e**log_est*1000*SCALE_FACTOR
dollar_hi = np.e**log_est*1000*SCALE_FACTOR
dollar_lo = np.e**log_est*1000*SCALE_FACTOR

#rounding the dollar value to nearest thousand
rounded_est = np.around(rounded_est, -3)
rounded_hi = np.around(rounded_hi, -3)
rounded_lo = np.around(rounded_lo, -3)


print(f'The estimated property value is {rounded_est}.')
print(f'At {conf}% confidence the valuation range is')
print(f'USD {rounded_lo} at the lower end to USD {rounded_hi} at the high end.')

The estimated property value is 918000.0.
At 68% confidence the valuation range is
USD 918000.0 at the lower end to USD 918000.0 at the high end.


In [28]:
#function for the above pythn code
def get_dollar_estimate(rm, ptratio, chas=False, large_range=True):
    """Estimate the price of a property in Boston.
    
    Keyword arguments:
    rm -- number of rooms in the property.
    ptratio -- number of students per teacher in the classroom for the school in the area.
    chas -- True if the property is next to the river, False otherwise.
    large_range -- True for a 95% prediction interval, False for a 68% interval.
    
    """
    
    
    if rm < 1 or ptratio < 1:
        print('That is unrealistic. Try again.')
        return

    log_est, upper, lower, conf = get_log_estimate(rm, 
                                                   students_per_classroom=ptratio, 
                                                   next_to_river=chas, 
                                                   high_confidence=large_range)

    # Convert to today's dollars
    dollar_est = np.e**log_est * 1000 * SCALE_FACTOR
    dollar_hi = np.e**upper * 1000 * SCALE_FACTOR
    dollar_low = np.e**lower * 1000 * SCALE_FACTOR

    # Round the dollar values to nearest thousand
    rounded_est = np.around(dollar_est, -3)
    rounded_hi = np.around(dollar_hi, -3)
    rounded_low = np.around(dollar_low, -3)

    print(f'The estimated property value is {rounded_est}.')
    print(f'At {conf}% confidence the valuation range is')
    print(f'USD {rounded_low} at the lower end to USD {rounded_hi} at the high end.')

In [29]:
get_dollar_estimate(6,-60,chas=True)

That is unrealistic. Try again.


In [35]:
import boston_valuation as val
val.get_dollar_estimate(6,12,True)

ValueError: Found array with 0 sample(s) (shape=(0, 11)) while a minimum of 1 is required.